In [1]:
%%time
# core importations
from annotationframeworkclient import FrameworkClient
import pandas as pd

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

# ----------CORE DATA GATHERING FUCNTIONS----------

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to get nucleus table entry using root id
def getNuc(root):
    client = FrameworkClient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"pt_root_id": [root]},
                                            materialization_version = mat_vers)
    return nuc_df 

# defines function to get synapse table entry using root id
def getSyn(root):
    client = FrameworkClient("flywire_fafb_production")
    mat_vers = max(client.materialize.get_versions())
    syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root]})
    return len(syn_df)

# defines function to get edit data
def getEdits(root):
    client = FrameworkClient("flywire_fafb_production")
    changes_dict = client.chunkedgraph.get_change_log(root)
    return changes_dict   

# defines function to build dataframe using root id list
def dfBuilder(root_list):
    data_list = []
    
    for i in root_list:
        sub_list = []
        sub_list.append(i) #adds root#
        nuc_df = getNuc(i) #pulls nucleus table#
        change_dict = getEdits(i) #pulls changelog#
        sub_list.append(nuc_df.loc[0,'id']) #adds nucleus id #
        sub_list.append(coordConvert(nuc_df.loc[0,'pt_position'])) #adds coordinates#
        sub_list.append(nuc_df.loc[0,'volume']) #adds volume#
        sub_list.append(getSyn(i)) #adds synapses#
        sub_list.append(change_dict['n_splits']) #adds splits# 
        sub_list.append(change_dict['n_mergers']) #adds mergers#
        sub_list.append((sub_list[-1] + sub_list[-2])) #adds edits#
#         sub_list.append(change_dict['user_info']) #adds editors, TEMPORARILY DISABLED#
        data_list.append(sub_list) #adds entry to data_list#

    # converts data_list into dataframe
    df = pd.DataFrame(data_list,columns = ['root_id','nucleus_id','nucleus_coordinates','volume','synapses',
                                           'mergers','splits','edits'])
    return df
   

Wall time: 1.54 s


In [12]:


df = pd.DataFrame(columns = ['root_id', 'nucleus_id', 'nucleus_coordinates', 'volume', 'synapses', 'mergers', 'splits', 'edits'])

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(dcc.Input(id='input_field', type='text', placeholder='Root ID(s)')), #formatting for input box#
#     html.Div(dcc.Checklist(
#         options=[
#             {'label': 'Nucleus ID'},
#             {'label': 'Nucleus Coordinates'},
#             {'label': 'Volume'},
#             {'label': 'Synapses'},
#             {'label': 'Mergers'},
#             {'label': 'Splits'},
#             {'label': 'Edits'}
#         ]
#     )),
    html.Button('Submit', id='submit_button', n_clicks=0), #formatting for button#
    html.Div(dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
#         style_data={
#             'color': 'white',
#             'backgroundColor': 'rgb(50, 50, 50)'
#         },
#         style_data_conditional=[
#             {
#             'if': {'row_index': 'odd'},
#             'backgroundColor': 'rgb(75, 75, 75)',
#             }
#         ],
#         style_header={
#             'backgroundColor': 'rgb(30, 30, 30)',
#             'color': 'white',
#             'fontWeight': 'bold'
#         },
#         style_as_list_view=True
    ))  
])

@app.callback(
    Output('table', 'data'), #defines output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'), #defines trigger as button press (change in the state of the 'n_clicks' aspect of 'submit_button')# 
    State('input_field', 'value') #defines state as the value of 'input_field'#
)
#defines function that executes when value of 'n_clicks' changes#
#takes value of 'State' as argument, returns output to 'Output' location above#
def update_output(n_clicks, roots):
    root_list = str(roots).split(",")
    root_list = [int(x.strip(' ')) for x in root_list]
    df = dfBuilder(root_list)
    df.head()
    return df.to_dict('records')

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2021 17:11:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:11:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:11:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:11:07] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:11:07] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:11:07] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [20/Oct/2021 17:11:07] "POST /_dash-update-component HTTP/1.1" 500 -
<timed exec>:51: FutureWarning:

'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.

<timed exec>:56: FutureWarning:

'pyarrow.deserialize' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.

127.0.0.1 - - [20/Oct/2021 17:11:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:11:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:13:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:13:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:13:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:13:17] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:13:18] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1

...

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [20/Oct/2021 17:14:24] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [20/Oct/2021 17:14:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:14:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:14:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:14:28] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:14:28] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 17:14:28] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Moose\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Moose\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [20/Oct/2021 17:14:28] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [20/Oct/2021 17:14:54] "POST /_dash-update-component HTTP/1.1" 200 -
